In [6]:
# load data and pre-processing
from lib.SESSC import SESSC
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, balanced_accuracy_score


f = np.load('wpbc.npz')
data, label = f['con_data'], f['label']
print(data.shape, label.shape)
trains, tests = f['trains'], f['tests']  # load train test split 
x_train, y_train = data[trains[0]], label[trains[0]]
x_test, y_test = data[tests[0]], label[tests[0]]

# pre-processing by z-score
ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.transform(x_test)


(1055, 41) (1055,)


In [7]:
# hyper-parameter

n_rules = 30
order = 0


In [8]:
# FCM-LSE
fcm_lse = TSK_FS(n_cluster=n_rules, order=order, cluster='fcm', classifier='ridge', _cls_alpha=0.0001, _cluster_scale=0.1)
fcm_lse.fit(x_train, y_train)
pred = fcm_lse.predict(x_test)
acc = accuracy_score(y_test, pred)
bca = balanced_accuracy_score(y_test, pred)
print('[Method: %10s] ACC: %.4f, BCA: %.4f' % ('FCM-LSE', acc, bca))

[Method:    FCM-LSE] ACC: 0.6719, BCA: 0.5140


In [9]:
# EWFCM-LSE
essc_lse = TSK_FS(n_cluster=n_rules, order=order, cluster='essc', classifier='ridge',
                 _cls_alpha=0.001, _cluster_scale=100, _cluster_eta=0, _cluster_gamma=100)
essc_lse.fit(x_train, y_train)
pred = essc_lse.predict(x_test)
acc = accuracy_score(y_test, pred)
bca = balanced_accuracy_score(y_test, pred)
print('[Method: %10s]ACC: %.4f, BCA: %.4f' % ('EWFCM-LSE', acc, bca))

[Method:  EWFCM-LSE]ACC: 0.7792, BCA: 0.7348


In [10]:
# ESSC-LSE
essc_lse = TSK_FS(n_cluster=n_rules, order=order, cluster='essc', classifier='ridge',
                 _cls_alpha=0.001, _cluster_scale=1, _cluster_eta=0.5, _cluster_gamma=100)
essc_lse.fit(x_train, y_train)
pred = essc_lse.predict(x_test)
acc = accuracy_score(y_test, pred)
bca = balanced_accuracy_score(y_test, pred)
print('[Method: %10s]ACC: %.4f, BCA: %.4f' % ('ESSC-LSE', acc, bca))

[Method:   ESSC-LSE]ACC: 0.7823, BCA: 0.7395


In [11]:
# SESSC
sessc = SESSC_Raw_Pred(n_cluster=30, eta=0.3, gamma=10, beta=10)
sessc.fit(x_train, y_train)
pred = sessc.predict(x_test)
acc = accuracy_score(y_test, pred)
bca = balanced_accuracy_score(y_test, pred)
print('[Method: %10s]ACC: %.4f, BCA: %.4f' % ('SESSC', acc, bca))


[Method:      SESSC]ACC: 0.6688, BCA: 0.5277


In [12]:
# SESSC-LSE
sessc_lse = TSK_FS(n_cluster=n_rules, order=order, cluster='sessc', classifier='ridge',
                 _cls_alpha=1, _cluster_scale=10, _cluster_eta=0.3, _cluster_gamma=100, _cluster_beta=10)
sessc_lse.fit(x_train, y_train)
pred = sessc_lse.predict(x_test)
acc = accuracy_score(y_test, pred)
bca = balanced_accuracy_score(y_test, pred)
print('[Method: %10s]ACC: %.4f, BCA: %.4f' % ('SESSC-LSE', acc, bca))


[Method:  SESSC-LSE]ACC: 0.8297, BCA: 0.7889
